In [ ]:
!pip install "dask[complete]"
!pip install pyarrow pandas
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.7/241.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path
import os
import datetime
from datetime import timedelta
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_colwidth', None)


In [ ]:
# Read the parquet file using Dask
ddf = dd.read_parquet('/content/drive/MyDrive/merged_data.parquet')
df = ddf.compute()
df.head()

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334fec1f741f658bfcbc1a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f78329e964152de23684,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c44216c782d07183742,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607a33d6b0efdeb592bac,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd590da6421ddfd950b810,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b3385ddb7b375241f15,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c9fe44c5388cf45b807,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a35bd21fceddaad0a1fb,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03


In [ ]:
# from merged data select the data where 'VALUEDATE'>= '2023-12-03'& 'VALUEDATE' <= '2023-12-07'

filtered_df = df[(df['VALUEDATE'] >= '2023-12-03') & (df['VALUEDATE'] <= '2023-12-07')]
filtered_df


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334fec1f741f658bfcbc1a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295a077d17e40491d1c28,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018c4ee971bf185b4b362,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f78329e964152de23684,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5ddec59322ebc61e86d,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c44216c782d07183742,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607a33d6b0efdeb592bac,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce309aef278bfed89856,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd590da6421ddfd950b810,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b3385ddb7b375241f15,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e300182370090564f0cae691746,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c9fe44c5388cf45b807,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a35bd21fceddaad0a1fb,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855088,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794685820,VND,4750.0,4750.0,110000,...,<NA>,N,1|2,98be8d28c01235ba77ff244ca6035dd86086b1ca28ca801a4fca515f57b3b9d6,MOB,HOI SO SHB,A2,VU THI YEN,NaN,2023-12-07
855089,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734517,VND,-3500.0,-3500.0,110000,...,<NA>,N,1|2,4dc1194ff047ca5307f6848f4b69aa562c00e6022e5413205404eb622c803a6a,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-07
855090,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734516,VND,900.0,900.0,110000,...,<NA>,Y,1|1,983fc2ba457d77883ea2a5160e77b81ffb98dc6e325abec2907a4ed8e43ec90c,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-07
855091,b4bb40c42e955a98f1641564f944c8b630b33307e680428e76fb37322d71caf1,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794734518,VND,-850.0,-850.0,110000,...,<NA>,N,1|2,38213e45e4a821d58cc7f7ca3294ba14479168f67bf22ea53ca5a15e2eb8bc60,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-07


In [ ]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4000821 entries, 0 to 855092
Data columns (total 26 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   CUST_CUSTNO            string        
 1   ACC_BUSINESSTYPE       string        
 2   ACC_ACCNO              string        
 3   ACC_BUSINESSNO         string        
 4   ACC_CURRENCYISO        string        
 5   BUSINESSNO_TRANS       string        
 6   FK_CURRENCY            string        
 7   AMOUNT                 float64       
 8   AMOUNTORIG             float64       
 9   BRANCH_OFFICE          string        
 10  CONTRA_ACCNO           string        
 11  CONTRA_ZIP             string        
 12  CONTRA_NAME            string        
 13  CSHYN                  string        
 14  REASON1                string        
 15  REASON2                string        
 16  REASON4                string        
 17  TR_FLAG_01             string        
 18  ANALYTICAL_TRANS_CODE  strin

In [ ]:
for column in filtered_df.columns:
    if pd.api.types.is_string_dtype(filtered_df[column]):
       filtered_df[column] = filtered_df[column].fillna('')
    elif pd.api.types.is_numeric_dtype(filtered_df[column]):
       filtered_df[column] = filtered_df[column].fillna(0)

In [ ]:
valeurs_manquantes = filtered_df.isnull().sum()
valeurs_manquantes

CUST_CUSTNO              0
ACC_BUSINESSTYPE         0
ACC_ACCNO                0
ACC_BUSINESSNO           0
ACC_CURRENCYISO          0
BUSINESSNO_TRANS         0
FK_CURRENCY              0
AMOUNT                   0
AMOUNTORIG               0
BRANCH_OFFICE            0
CONTRA_ACCNO             0
CONTRA_ZIP               0
CONTRA_NAME              0
CSHYN                    0
REASON1                  0
REASON2                  0
REASON4                  0
TR_FLAG_01               0
ANALYTICAL_TRANS_CODE    0
TR_SP_01                 0
TR_SP_02                 0
TR_SP_03                 0
TR_SP_04                 0
TR_SP_05                 0
TR_SP_10                 0
VALUEDATE                0
dtype: int64

In [ ]:
label_encoders = {}
list_columns = ['ACC_BUSINESSTYPE', 'ACC_CURRENCYISO', 'TR_SP_01', 'TR_SP_02', 'TR_SP_03', 'TR_SP_04', 'TR_SP_05', 'TR_SP_10']

In [ ]:
for column in list_columns:
    le = LabelEncoder()
    filtered_df[column] = le.fit_transform(filtered_df[column])
    label_encoders[column] = le

In [ ]:

string_columns = [col for col in filtered_df.columns if pd.api.types.is_string_dtype(filtered_df[col])]
string_columns_without_list = [col for col in string_columns if col not in list_columns]
string_columns_without_list

['CUST_CUSTNO',
 'ACC_ACCNO',
 'ACC_BUSINESSNO',
 'BUSINESSNO_TRANS',
 'FK_CURRENCY',
 'BRANCH_OFFICE',
 'CONTRA_ACCNO',
 'CONTRA_ZIP',
 'CONTRA_NAME',
 'CSHYN',
 'REASON1',
 'REASON2',
 'REASON4',
 'TR_FLAG_01',
 'ANALYTICAL_TRANS_CODE']

In [ ]:
label_encoders = {}

for column in string_columns_without_list:
    le = LabelEncoder()
    filtered_df[column] = le.fit_transform(filtered_df[column])
    label_encoders[column] = le

In [ ]:
filtered_df

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,29286,2,19911,188,11,549587,14,-650.0,-650.0,0,...,0,0,13,3184591,9,61,11,0,0,2023-12-03
1,29286,2,19911,188,11,36324,14,-34.0,-34.0,0,...,0,0,13,2512309,9,61,11,0,0,2023-12-03
2,36020,2,352121,188,11,298901,14,120000.0,120000.0,0,...,0,1,12,479432,10,61,11,0,0,2023-12-03
3,108389,2,61870,188,11,246470,14,-62.0,-62.0,0,...,0,0,13,3504509,9,61,11,0,0,2023-12-03
4,118275,2,261401,188,11,88478,14,-1500.0,-1500.0,0,...,0,0,13,1187641,9,61,11,0,0,2023-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855088,235987,0,8534,188,0,3254638,14,4750.0,4750.0,0,...,0,0,13,2131891,9,61,0,62155,0,2023-12-07
855089,235987,0,8534,188,0,3303335,14,-3500.0,-3500.0,0,...,0,0,13,1086152,9,61,11,0,0,2023-12-07
855090,235987,0,8534,188,0,3303334,14,900.0,900.0,0,...,0,1,12,2125157,10,61,11,0,0,2023-12-07
855091,235987,0,8534,188,0,3303336,14,-850.0,-850.0,0,...,0,0,13,784275,9,61,11,0,0,2023-12-07


In [ ]:

# Apply FP-Growth algorithm
fp_growth_results = fpgrowth(filtered_df.applymap(lambda x: 1 if x else 0) ,min_support=0.9, use_colnames=True)

In [ ]:
fp_growth_results

,support,itemsets
0,1.000000,(VALUEDATE)
1,1.000000,(TR_SP_01)
2,1.000000,(BUSINESSNO_TRANS)
3,1.000000,(ACC_BUSINESSNO)
4,0.999999,(CUST_CUSTNO)
...,...,...
106490,0.903434,"(ANALYTICAL_TRANS_CODE, CONTRA_ACCNO, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, ACC_BUSINESSNO, AMOUNTORIG, FK_CURRENCY, VALUEDATE, TR_SP_03, AMOUNT, TR_SP_02, TR_SP_04)"
106491,0.903434,"(ANALYTICAL_TRANS_CODE, CONTRA_ACCNO, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, TR_SP_04, ACC_BUSINESSNO, AMOUNTORIG, FK_CURRENCY, BUSINESSNO_TRANS, AMOUNT, TR_SP_02, TR_SP_03)"
106492,0.903434,"(ANALYTICAL_TRANS_CODE, CONTRA_ACCNO, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, TR_SP_04, ACC_BUSINESSNO, FK_CURRENCY, VALUEDATE, BUSINESSNO_TRANS, AMOUNT, TR_SP_02, TR_SP_03)"
106493,0.903434,"(ANALYTICAL_TRANS_CODE, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, TR_SP_04, ACC_BUSINESSNO, AMOUNTORIG, FK_CURRENCY, VALUEDATE, BUSINESSNO_TRANS, AMOUNT, TR_SP_02, TR_SP_03)"


In [ ]:
# prompt: slect itemsets that have CUST_CUSTNO and AMOUNT combinations an calculate frequency

filtered_itemsets = fp_growth_results[fp_growth_results['itemsets'].apply(lambda x: 'CUST_CUSTNO' in x and 'AMOUNT' in x)]
filtered_itemsets


,support,itemsets
1034,0.995715,"(AMOUNT, CUST_CUSTNO)"
1051,0.995715,"(VALUEDATE, AMOUNT, CUST_CUSTNO)"
1052,0.995715,"(AMOUNT, TR_SP_01, CUST_CUSTNO)"
1053,0.995715,"(BUSINESSNO_TRANS, AMOUNT, CUST_CUSTNO)"
1054,0.995715,"(AMOUNT, ACC_BUSINESSNO, CUST_CUSTNO)"
...,...,...
106490,0.903434,"(ANALYTICAL_TRANS_CODE, CONTRA_ACCNO, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, ACC_BUSINESSNO, AMOUNTORIG, FK_CURRENCY, VALUEDATE, TR_SP_03, AMOUNT, TR_SP_02, TR_SP_04)"
106491,0.903434,"(ANALYTICAL_TRANS_CODE, CONTRA_ACCNO, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, TR_SP_04, ACC_BUSINESSNO, AMOUNTORIG, FK_CURRENCY, BUSINESSNO_TRANS, AMOUNT, TR_SP_02, TR_SP_03)"
106492,0.903434,"(ANALYTICAL_TRANS_CODE, CONTRA_ACCNO, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, TR_SP_04, ACC_BUSINESSNO, FK_CURRENCY, VALUEDATE, BUSINESSNO_TRANS, AMOUNT, TR_SP_02, TR_SP_03)"
106493,0.903434,"(ANALYTICAL_TRANS_CODE, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, TR_SP_04, ACC_BUSINESSNO, AMOUNTORIG, FK_CURRENCY, VALUEDATE, BUSINESSNO_TRANS, AMOUNT, TR_SP_02, TR_SP_03)"


In [ ]:
# prompt: select from filtered itemsets the higher of support

itemsets=filtered_itemsets.loc[filtered_itemsets['support'] == filtered_itemsets['support'].max()]

itemsets

,support,itemsets
1034,0.995715,"(AMOUNT, CUST_CUSTNO)"
1051,0.995715,"(VALUEDATE, AMOUNT, CUST_CUSTNO)"
2091,0.995715,"(AMOUNT, AMOUNTORIG, CUST_CUSTNO)"
2095,0.995715,"(VALUEDATE, AMOUNT, AMOUNTORIG, CUST_CUSTNO)"


In [ ]:
# Build association rules
rules = association_rules(fp_growth_results, metric="confidence", min_threshold=1) # Use fp_growth_results instead of itemsets

# Display the association rules
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(TR_SP_01),(VALUEDATE),1.000000,1.000000,1.000000,1.0,1.000000,0.000000,inf,0.000000
1,(BUSINESSNO_TRANS),(VALUEDATE),1.000000,1.000000,1.000000,1.0,1.000000,0.000000,inf,0.000000
2,"(BUSINESSNO_TRANS, TR_SP_01)",(VALUEDATE),1.000000,1.000000,1.000000,1.0,1.000000,0.000000,inf,0.000000
3,(ACC_BUSINESSNO),(VALUEDATE),1.000000,1.000000,1.000000,1.0,1.000000,0.000000,inf,0.000000
4,"(BUSINESSNO_TRANS, ACC_BUSINESSNO)",(VALUEDATE),0.999999,1.000000,0.999999,1.0,1.000000,0.000000,inf,0.000000
...,...,...,...,...,...,...,...,...,...,...
356602,"(ANALYTICAL_TRANS_CODE, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, ACC_BUSINESSNO, AMOUNTORIG, FK_CURRENCY, TR_SP_03, AMOUNT, TR_SP_02, TR_SP_04)","(VALUEDATE, BUSINESSNO_TRANS, CONTRA_ACCNO)",0.903434,0.999999,0.903434,1.0,1.000001,0.000001,inf,0.000016
356603,"(ANALYTICAL_TRANS_CODE, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, ACC_BUSINESSNO, FK_CURRENCY, VALUEDATE, TR_SP_03, AMOUNT, TR_SP_02, TR_SP_04)","(BUSINESSNO_TRANS, CONTRA_ACCNO, AMOUNTORIG)",0.903434,0.995714,0.903434,1.0,1.004304,0.003872,inf,0.044380
356604,"(ANALYTICAL_TRANS_CODE, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, ACC_BUSINESSNO, FK_CURRENCY, BUSINESSNO_TRANS, AMOUNT, TR_SP_02, TR_SP_04, TR_SP_03)","(VALUEDATE, CONTRA_ACCNO, AMOUNTORIG)",0.903434,0.995715,0.903434,1.0,1.004304,0.003872,inf,0.044378
356605,"(ANALYTICAL_TRANS_CODE, TR_SP_01, CUST_CUSTNO, ACC_ACCNO, REASON1, ACC_BUSINESSNO, AMOUNTORIG, FK_CURRENCY, TR_SP_03, TR_SP_02, TR_SP_04)","(VALUEDATE, BUSINESSNO_TRANS, CONTRA_ACCNO, AMOUNT)",0.903434,0.995714,0.903434,1.0,1.004304,0.003872,inf,0.044380


In [ ]:
# prompt: # prompt: select the rules that have 'CUST_CUTNO' and 'AMOUNT' in the antecedents select the best metrics and count of number item

filtered_rules = rules[(rules['antecedents'].apply(lambda x: 'CUST_CUSTNO' in x)) & (rules['consequents'].apply(lambda x: 'AMOUNT' in x))]
filtered_rules = filtered_rules.sort_values(by=['confidence', 'support'], ascending=False)
filtered_rules['number_of_items'] = filtered_rules['antecedents'].apply(lambda x: len(x))
filtered_rules = filtered_rules[filtered_rules['number_of_items'] == 2]
filtered_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,number_of_items
1089,"(AMOUNTORIG, CUST_CUSTNO)",(AMOUNT),0.995715,0.995716,0.995715,1.0,1.004303,0.004266,inf,0.999825,2
1094,"(AMOUNTORIG, CUST_CUSTNO)","(VALUEDATE, AMOUNT)",0.995715,0.995716,0.995715,1.0,1.004303,0.004266,inf,0.999825,2


In [ ]:
# prompt: # prompt: select the rules that have 'CUST_CUTNO' and 'AMOUNT' in the antecedents select the best metrics and count of number item

filtered_rules = rules[(rules['antecedents'].apply(lambda x: 'CUST_CUSTNO' in x)) & (rules['consequents'].apply(lambda x: 'AMOUNT' in x))]
filtered_rules = filtered_rules.sort_values(by=['confidence', 'support'], ascending=False)
filtered_rules['number_of_items'] = filtered_rules['antecedents'].apply(lambda x: len(x))
filtered_rules = filtered_rules[filtered_rules['number_of_items'] == 3]
filtered_rules.drop_duplicates(subset=['antecedents', 'consequents'], inplace=True)
filtered_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,number_of_items
1091,"(VALUEDATE, AMOUNTORIG, CUST_CUSTNO)",(AMOUNT),0.995715,0.995716,0.995715,1.0,1.004303,0.004266,inf,0.999825,3
1096,"(AMOUNTORIG, TR_SP_01, CUST_CUSTNO)",(AMOUNT),0.995715,0.995716,0.995715,1.0,1.004303,0.004266,inf,0.999767,3
1101,"(AMOUNTORIG, TR_SP_01, CUST_CUSTNO)","(VALUEDATE, AMOUNT)",0.995715,0.995716,0.995715,1.0,1.004303,0.004266,inf,0.999767,3
1104,"(BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO)",(AMOUNT),0.995715,0.995716,0.995715,1.0,1.004303,0.004266,inf,0.999767,3
1109,"(BUSINESSNO_TRANS, AMOUNTORIG, CUST_CUSTNO)","(VALUEDATE, AMOUNT)",0.995715,0.995716,0.995715,1.0,1.004303,0.004266,inf,0.999767,3
1120,"(ACC_BUSINESSNO, AMOUNTORIG, CUST_CUSTNO)",(AMOUNT),0.995715,0.995716,0.995715,1.0,1.004303,0.004266,inf,0.999708,3
1125,"(ACC_BUSINESSNO, AMOUNTORIG, CUST_CUSTNO)","(VALUEDATE, AMOUNT)",0.995715,0.995716,0.995715,1.0,1.004303,0.004266,inf,0.999708,3
1216,"(CONTRA_ACCNO, AMOUNTORIG, CUST_CUSTNO)",(AMOUNT),0.995714,0.995716,0.995714,1.0,1.004303,0.004266,inf,0.999533,3
1221,"(CONTRA_ACCNO, AMOUNTORIG, CUST_CUSTNO)","(VALUEDATE, AMOUNT)",0.995714,0.995716,0.995714,1.0,1.004303,0.004266,inf,0.999533,3
1344,"(ACC_ACCNO, AMOUNTORIG, CUST_CUSTNO)",(AMOUNT),0.995710,0.995716,0.995710,1.0,1.004303,0.004266,inf,0.998660,3


In [ ]:
# prompt: select the rules that have 'CUST_CUTNO' and 'AMOUNT' and the best metrics value

best_rules = rules[
    (rules['antecedents'].apply(lambda x: 'CUST_CUSTNO' in x)) &
    (rules['consequents'].apply(lambda x: 'AMOUNT' in x))
]

best_rules = best_rules[best_rules['confidence'] == best_rules['confidence'].max()]
best_rules = best_rules[best_rules['lift'] == best_rules['lift'].max()]
best_rules = best_rules[best_rules['leverage'] == best_rules['leverage'].max()]
best_rules = best_rules[best_rules['zhangs_metric'] == best_rules['zhangs_metric'].max()]
best_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
231277,"(ACC_CURRENCYISO, CUST_CUSTNO, ACC_ACCNO, AMOUNTORIG, TR_SP_04)","(BUSINESSNO_TRANS, CONTRA_ACCNO, ACC_BUSINESSTYPE, AMOUNT)",0.912629,0.979499,0.912629,1.0,1.02093,0.01871,inf,0.234647
231279,"(CUST_CUSTNO, ACC_ACCNO, ACC_BUSINESSTYPE, AMOUNTORIG, TR_SP_04)","(BUSINESSNO_TRANS, CONTRA_ACCNO, ACC_CURRENCYISO, AMOUNT)",0.912629,0.979499,0.912629,1.0,1.02093,0.01871,inf,0.234647
231361,"(ACC_CURRENCYISO, CUST_CUSTNO, ACC_ACCNO, AMOUNTORIG, VALUEDATE, TR_SP_04)","(BUSINESSNO_TRANS, CONTRA_ACCNO, ACC_BUSINESSTYPE, AMOUNT)",0.912629,0.979499,0.912629,1.0,1.02093,0.01871,inf,0.234647
231366,"(CUST_CUSTNO, ACC_ACCNO, ACC_BUSINESSTYPE, AMOUNTORIG, VALUEDATE, TR_SP_04)","(BUSINESSNO_TRANS, CONTRA_ACCNO, ACC_CURRENCYISO, AMOUNT)",0.912629,0.979499,0.912629,1.0,1.02093,0.01871,inf,0.234647
231371,"(ACC_CURRENCYISO, CUST_CUSTNO, ACC_ACCNO, AMOUNTORIG, TR_SP_04)","(CONTRA_ACCNO, ACC_BUSINESSTYPE, VALUEDATE, BUSINESSNO_TRANS, AMOUNT)",0.912629,0.979499,0.912629,1.0,1.02093,0.01871,inf,0.234647
231373,"(CUST_CUSTNO, ACC_ACCNO, ACC_BUSINESSTYPE, AMOUNTORIG, TR_SP_04)","(CONTRA_ACCNO, ACC_CURRENCYISO, VALUEDATE, BUSINESSNO_TRANS, AMOUNT)",0.912629,0.979499,0.912629,1.0,1.02093,0.01871,inf,0.234647


In [ ]:


# Identify records that match the antecedents
rule_antecedents = ['VALUEDATE', 'AMOUNTORIG', 'CUST_CUSTNO']
rule_consequent = 'AMOUNT'
matched_records = filtered_df[filtered_df[rule_antecedents].notnull().all(axis=1)]
# Display the matched records
matched_records


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
0,1655bf1b772d90df34c51ba9240f683d3847700b330295...,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391980769,VND,-650.0,-650.0,110000,...,<NA>,N,1|2,e4180c1cc088ebea7f7a882144f55159a8dddad215334f...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
1,1655bf1b772d90df34c51ba9240f683d3847700b330295...,CASA,0c643de92622a5f3bdb856d4471510b234be53fb0ab018...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391467506,VND,-34.0,-34.0,110000,...,<NA>,N,1|2,b403758c4a7b9fbc934464efaaaab53c7e24f0b68891f7...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
2,1b781f92156f274627a131e707e7a890548aea921b84f5...,CASA,df774ebe64129d3a6cf3af75e7de8d01ca85e92b2f2c1c...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391730083,VND,120000.0,120000.0,110000,...,<NA>,Y,1|1,22527ba60412b7170a3dfcd831805f966b244b354d1607...,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-03
3,53048c1afff7f2cff6e1469fd1659f99ec93696df7a3ce...,CASA,2715ec2c8fc66cc32a6accc328d63843fb0dcd5565cd59...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391677652,VND,-62.0,-62.0,110000,...,<NA>,N,1|2,fb05380821892df14964a30cbe3fd795e0cb225c8da13b...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
4,5abad7746ef69e998247ba7848ca6224dd106e30018237...,CASA,a5dc6ed17504c973d61d1ea548eb1945ae698847f3cb5c...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,VND,2023120391519660,VND,-1500.0,-1500.0,110000,...,<NA>,N,1|2,54ff7b7a4f04b0a6e7de5abd7a4473ec7831e5579431a3...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855088,b4bb40c42e955a98f1641564f944c8b630b33307e68042...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2023120794685820,VND,4750.0,4750.0,110000,...,<NA>,N,1|2,98be8d28c01235ba77ff244ca6035dd86086b1ca28ca80...,MOB,HOI SO SHB,A2,VU THI YEN,NaN,2023-12-07
855089,b4bb40c42e955a98f1641564f944c8b630b33307e68042...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2023120794734517,VND,-3500.0,-3500.0,110000,...,<NA>,N,1|2,4dc1194ff047ca5307f6848f4b69aa562c00e6022e5413...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-07
855090,b4bb40c42e955a98f1641564f944c8b630b33307e68042...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2023120794734516,VND,900.0,900.0,110000,...,<NA>,Y,1|1,983fc2ba457d77883ea2a5160e77b81ffb98dc6e325abe...,NET,HOI SO SHB,FP,<NA>,NaN,2023-12-07
855091,b4bb40c42e955a98f1641564f944c8b630b33307e68042...,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af...,<NA>,2023120794734518,VND,-850.0,-850.0,110000,...,<NA>,N,1|2,38213e45e4a821d58cc7f7ca3294ba14479168f67bf22e...,MOB,HOI SO SHB,FP,<NA>,NaN,2023-12-07


**The rule indicates that knowing the date of a transaction, the original amount, and the customer ID helps predict the actual traded amount. The threshold helps identify transactions that deviate significantly**

In [ ]:
# prompt: # Calculate expected AMOUNTORING

expected_amountorig = matched_records['AMOUNTORIG'].mean()
print(f"Expected AMOUNTORIG: {expected_amountorig}")


Expected AMOUNTORIG: 1131.4447831257644


In [ ]:
# Calculate deviations
matched_records['Deviation'] = matched_records['AMOUNT'] - expected_amountorig

In [ ]:
matched_records['Deviation']

0          -1781.444783
1          -1165.444783
2         118868.555217
3          -1193.444783
4          -2631.444783
              ...      
855088      3618.555217
855089     -4631.444783
855090      -231.444783
855091     -1981.444783
855092      1368.555217
Name: Deviation, Length: 4000821, dtype: float64

In [ ]:

# Set a threshold for anomalies
threshold = 2 * matched_records['Deviation'].std()

In [ ]:
threshold

20590541.840384312

In [ ]:

# Flag anomalies
matched_records['Is_Anomaly'] = matched_records['Deviation'].abs() > threshold

# Display the results
matched_records[matched_records['Is_Anomaly'] == True]


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE,Deviation,Is_Anomaly
621230,282de1ab25e2fbfa0dbae88dfd3dbb56beef6f55565e8475f473e7ae9dd6f8af,LOAN,ec38214fc782e37f206c5a4138fa95e4b30a1f2595f20f4fa4f99b1d249c3f20,60b05918423d87f5c9169c012932b5ee6f9b2206bb9ae2e75700cf92ab57ed64,VND,2023120392080660,VND,1.657137e+08,1.657137e+08,111900,...,21|2,27e9a23faf96510e488855c9ef9e4df6042706ae4a562376c8542ded44da5727,BRC,CN KINH DO,LN,<NA>,NaN,2023-12-03,1.657126e+08,True
621231,282de1ab25e2fbfa0dbae88dfd3dbb56beef6f55565e8475f473e7ae9dd6f8af,LOAN,ec38214fc782e37f206c5a4138fa95e4b30a1f2595f20f4fa4f99b1d249c3f20,60b05918423d87f5c9169c012932b5ee6f9b2206bb9ae2e75700cf92ab57ed64,VND,2023120392080821,VND,-1.657137e+08,-1.657137e+08,111900,...,21|1,27e9a23faf96510e488855c9ef9e4df6042706ae4a562376c8542ded44da5727,BRC,CN KINH DO,LN,<NA>,NaN,2023-12-03,-1.657148e+08,True
621232,282de1ab25e2fbfa0dbae88dfd3dbb56beef6f55565e8475f473e7ae9dd6f8af,LOAN,49d8db4e79d2aed4361ad59de10e63fcd8dafa71000ecb1242420ba560bfbbc2,b608a86cf8182394f495c8f3ac4f041f32b5054b6d43c2cee9341e5d0dcf6c8b,VND,2023120392080706,VND,-1.194226e+08,-1.194226e+08,111900,...,21|1,7ef8335fa0b4220088d30f39cea739b958d03bb8a4b067666a0af7ade3100ace,BRC,CN KINH DO,LN,<NA>,NaN,2023-12-03,-1.194237e+08,True
621233,282de1ab25e2fbfa0dbae88dfd3dbb56beef6f55565e8475f473e7ae9dd6f8af,LOAN,49d8db4e79d2aed4361ad59de10e63fcd8dafa71000ecb1242420ba560bfbbc2,b608a86cf8182394f495c8f3ac4f041f32b5054b6d43c2cee9341e5d0dcf6c8b,VND,2023120392080766,VND,1.194226e+08,1.194226e+08,111900,...,21|2,7ef8335fa0b4220088d30f39cea739b958d03bb8a4b067666a0af7ade3100ace,BRC,CN KINH DO,LN,<NA>,NaN,2023-12-03,1.194215e+08,True
621234,282de1ab25e2fbfa0dbae88dfd3dbb56beef6f55565e8475f473e7ae9dd6f8af,LOAN,282dfc8a9d1ad358b1230b45156a415d91e800368e4974e4012088b25cfbaab7,eba2e6c3cf39e46f169feda3dacbf3a611de3a64419f55363054025d095ad84c,VND,2023120392080542,VND,1.310010e+08,1.310010e+08,111900,...,21|2,7cd68877e9ae7c9c3c68aab91e7b755a97b0d6e2d0f5c5b5631d167a74662659,BRC,CN KINH DO,LN,<NA>,NaN,2023-12-03,1.309998e+08,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842356,662cb8ee5ab14682d18ea8bee7b7627774687a79d07d71b697357a1461aa96c8,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794684601,VND,1.680000e+08,1.680000e+08,110301,...,2|2,42c32cf84f8622e17234a5f21635946c0b249f29d7375706449fac03981e46b2,BRC,PGD HA LONG/CN QUANG NINH,A2,TONG QUANG NGHIA,NaN,2023-12-07,1.679989e+08,True
842357,662cb8ee5ab14682d18ea8bee7b7627774687a79d07d71b697357a1461aa96c8,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794684602,VND,1.000000e+08,1.000000e+08,110313,...,2|2,60bfa48119494f4d8bf8414e600031caeb4dafc2fb4a8a2bfa40df25fa4e8d3f,BRC,PGD QUANG TRUONG/CN QUANG NINH,A2,TONG QUANG NGHIA,NaN,2023-12-07,9.999887e+07,True
847638,58fb7761353ffade5e940b32ce9840eab7c74dffe24f38fc66fc57eebd071856,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794697723,USD,2.241726e+07,9.232048e+05,111600,...,1|1,5aa56ec56259aa5350370d28e01f394b49e2dc874e5dcf5dad19b88cd02cb3db,BRC,CN VAN PHUC,OT,<NA>,NaN,2023-12-07,2.241613e+07,True
847639,58fb7761353ffade5e940b32ce9840eab7c74dffe24f38fc66fc57eebd071856,<NA>,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,<NA>,2023120794697724,USD,-2.241726e+07,-9.232048e+05,111600,...,1|1,5aa56ec56259aa5350370d28e01f394b49e2dc874e5dcf5dad19b88cd02cb3db,BRC,CN VAN PHUC,OT,<NA>,NaN,2023-12-07,-2.241839e+07,True


In [ ]:
# prompt: # Summarize Anomalies

# Group anomalies by date
anomaly_summary = matched_records[matched_records['Is_Anomaly'] == True].groupby('VALUEDATE')['AMOUNT'].agg(['sum', 'count'])

# Calculate total amount and number of anomalies for each date
anomaly_summary['Total Amount'] = anomaly_summary['sum']
anomaly_summary['Number of Anomalies'] = anomaly_summary['count']

# Display the anomaly summary
anomaly_summary

# Sort the anomaly summary by date
anomaly_summary = anomaly_summary.sort_values(by='VALUEDATE')

# Display the sorted anomaly summary
anomaly_summary


,sum,count,Total Amount,Number of Anomalies
VALUEDATE,,,,
2023-12-03,2.465377e+07,13,2.465377e+07,13
2023-12-04,1.415621e+09,910,1.415621e+09,910
2023-12-05,2.425424e+09,850,2.425424e+09,850
2023-12-06,6.791206e+08,592,6.791206e+08,592
2023-12-07,4.340204e+09,765,4.340204e+09,765


In [ ]:
# prompt: # Compare with Non-Anomalous Records

# Select non-anomalous records
non_anomalous_records = matched_records[matched_records['Is_Anomaly'] == False]

# Calculate summary statistics for non-anomalous records
non_anomalous_summary = non_anomalous_records.describe()

# Display the summary statistics
print(non_anomalous_summary)

# Compare the summary statistics of anomalous and non-anomalous records
print("Comparison of Anomalous and Non-Anomalous Records:")
print("- Average Amount (Anomalous):", matched_records[matched_records['Is_Anomaly'] == True]['AMOUNT'].mean())
print("- Average Amount (Non-Anomalous):", non_anomalous_records['AMOUNT'].mean())
print("- Standard Deviation (Anomalous):", matched_records[matched_records['Is_Anomaly'] == True]['AMOUNT'].std())
print("- Standard Deviation (Non-Anomalous):", non_anomalous_records['AMOUNT'].std())


             AMOUNT    AMOUNTORIG     TR_SP_10                      VALUEDATE  \
count  3.997691e+06  3.997691e+06  3745.000000                        3997691   
mean  -6.345923e+02 -6.343378e+02  5624.962021  2023-12-05 01:32:22.281687808   
min   -2.058849e+07 -4.900000e+08  4121.070000            2023-12-03 00:00:00   
25%   -5.000000e+02 -5.000000e+02  5418.250000            2023-12-04 00:00:00   
50%   -1.500000e+01 -1.500000e+01  5776.130000            2023-12-05 00:00:00   
75%    3.500000e+02  3.500000e+02  6035.940000            2023-12-06 00:00:00   
max    2.040000e+07  4.900000e+08  9476.080000            2023-12-07 00:00:00   
std    3.440107e+05  9.131017e+05   928.070270                            NaN   

          Deviation  
count  3.997691e+06  
mean  -1.766037e+03  
min   -2.058962e+07  
25%   -1.631445e+03  
50%   -1.146445e+03  
75%   -7.814448e+02  
max    2.039887e+07  
std    3.440107e+05  
Comparison of Anomalous and Non-Anomalous Records:
- Average Amount (Ano

In [ ]:
# prompt: from matched_records[matched_records['Is_Anomaly'] == True] display the top  accounts having the top AMOUNT

top_anomalies = matched_records[matched_records['Is_Anomaly'] == True].sort_values(by='AMOUNT', ascending=False).head(10)
top_anomalies[['CUST_CUSTNO', 'AMOUNT']]


,CUST_CUSTNO,AMOUNT
637793,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,2.500000e+09
930302,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,2.500000e+09
634959,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,2.500000e+09
930305,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,2.500000e+09
788587,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,2.000000e+09
213300,f82a9d25673478a95ae762bb6ee313385eb3c2da73da25326f071c72e2e2f326,2.000000e+09
87828,6567a448ec70441baea13fb4f8be538144ef16cb92fabd6c66292976152e54d6,2.000000e+09
748424,84da7ef354174252021e10431da64dae7b513b660a0bcb432054964d842b4cdf,2.000000e+09
788586,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,2.000000e+09
932266,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,1.500000e+09


In [ ]:
# prompt: display the account from anomalies having the top AMOUNT in one day

# Get the top anomaly account number
top_account = top_anomalies['CUST_CUSTNO'].iloc[0]

# Filter the anomalies to only include the top account
top_account_anomalies = top_anomalies[top_anomalies['CUST_CUSTNO'] == top_account]

# Display the top account anomalies
top_account_anomalies


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE,Deviation,Is_Anomaly
637793,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,CASA,ce22f0e8123ff8e3952704f3168c6e74784778ac22742744cfc9f8759596ad66,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120593114694,VND,2.500000e+09,2.500000e+09,110007,...,8|17,072ca6b3e5dd62ee1122cc1163c39f4ff53c63715c9f80082ca029f069eab496,BRC,TRUNG TAM QUAN LY VON,MM,<NA>,NaN,2023-12-05,2.499999e+09,True
930302,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,CASA,ce22f0e8123ff8e3952704f3168c6e74784778ac22742744cfc9f8759596ad66,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120492245810,VND,2.500000e+09,2.500000e+09,110001,...,7|41,8b4fba3f717f750b7f4e5a097d2f760d468d2bc36662ab3ed708662b0ad30ba7,BRC,TRUNG TAM KD VON,MM,<NA>,NaN,2023-12-04,2.499999e+09,True
634959,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,CASA,ce22f0e8123ff8e3952704f3168c6e74784778ac22742744cfc9f8759596ad66,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120593114581,VND,2.500000e+09,2.500000e+09,110001,...,8|17,1d7082659e987ec4acf99dda8864d11e9ef420faea8fd6fb94f313b6cc745952,BRC,TRUNG TAM KD VON,MM,<NA>,NaN,2023-12-05,2.499999e+09,True
930305,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,CASA,ce22f0e8123ff8e3952704f3168c6e74784778ac22742744cfc9f8759596ad66,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120492245813,VND,2.500000e+09,2.500000e+09,110007,...,7|41,ba70386c09f09e77162d580d5c73806aaa4bbcb615b4aa0d4b7ec1f0afc0101a,BRC,TRUNG TAM QUAN LY VON,MM,<NA>,NaN,2023-12-04,2.499999e+09,True
788587,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,CASA,ce22f0e8123ff8e3952704f3168c6e74784778ac22742744cfc9f8759596ad66,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120794761702,VND,2.000000e+09,2.000000e+09,110001,...,7|41,5d5bf354448be968c7a226498ce5d436773eb61830b0c3f972337214dcf6d9b2,BRC,TRUNG TAM KD VON,MM,<NA>,NaN,2023-12-07,1.999999e+09,True
788586,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,CASA,ce22f0e8123ff8e3952704f3168c6e74784778ac22742744cfc9f8759596ad66,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120794761700,VND,2.000000e+09,2.000000e+09,110007,...,7|41,3def01a6ac6048388c8a4e50ad12a231aee3e88505f6e876ab3b590ad22b21bd,BRC,TRUNG TAM QUAN LY VON,MM,<NA>,NaN,2023-12-07,1.999999e+09,True
932266,4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9,CASA,ce22f0e8123ff8e3952704f3168c6e74784778ac22742744cfc9f8759596ad66,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2023120492245895,VND,1.500000e+09,1.500000e+09,110007,...,8|17,1ed001a297cb880bc5160d3dde81dc74ccc103dc4aca2d81ca2828314413d4d5,BRC,TRUNG TAM QUAN LY VON,MM,<NA>,NaN,2023-12-04,1.499999e+09,True


In [ ]:
# prompt: count number of transaction of this account 4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9

# Count the number of transactions for the top account per time
top_account_transactions = filtered_df[filtered_df['CUST_CUSTNO'] == top_account].groupby('VALUEDATE')['AMOUNT'].count().reset_index(name='Number_of_Transactions')

# Display the top account transactions per time
top_account_transactions


,VALUEDATE,Number_of_Transactions
0,2023-12-04,13634
1,2023-12-05,1603
2,2023-12-06,1078
3,2023-12-07,912


In [ ]:
# prompt: # Further analysis by grouping anomalies by CUST_CUSTNO to see if certain customers have more anomalies

# Group anomalies by CUST_CUSTNO
grouped_anomalies = matched_records[matched_records['Is_Anomaly'] == True].groupby('CUST_CUSTNO')['AMOUNT'].agg(['sum', 'count'])

# Calculate total amount and number of anomalies for each customer
grouped_anomalies['Total Amount'] = grouped_anomalies['sum']
grouped_anomalies['Number of Anomalies'] = grouped_anomalies['count']

# Display the anomaly summary by customer
print(grouped_anomalies.sort_values(by='Number of Anomalies', ascending=False))

# Identify customers with the most anomalies
top_anomalous_customers = grouped_anomalies.sort_values(by='Number of Anomalies', ascending=False).head(10)
top_anomalous_customers

# Investigate the transactions of the top anomalous customer
top_anomalous_customer = top_anomalous_customers.index[0]
top_customer_transactions = filtered_df[filtered_df['CUST_CUSTNO'] == top_anomalous_customer]


                                                                           sum  \
CUST_CUSTNO                                                                      
4c9a21532c77256b35197dc53db54243139fbc39261024dd40a8445039c825c9  6.830307e+07   
6d681af72190882763ad5115560d38bb219881aa0c68f6915bf54bc9a3d6c254  7.384756e+09   
3445fd72e0c2224374abdce982eeb1faa3430b6a7812d620475d4ed630b87a30 -2.390032e+09   
be91ed7ac11d81ef2f701117c8148585fb59e88b2c4028974d52bf39440abee7 -6.916465e+08   
7f98d8ed6fb87414249c623f5878f66895bec80ce99579321d4e8007c778f1c3  7.450581e-09   
...                                                                        ...   
ed8ec90293dda95ea0dabec992f3973d841203ad1f31c8aa087fea01e8c616a0  2.690500e+07   
0a4ead3d77267bb4118b6efd8aa3ba580881d2539c0f53fc83179234b1d37624  1.000000e+08   
0b5023c23745b483f339dcb5e74edca09fedbe5277ff09a1088406320d03f83b -2.360110e+07   
0356e1b526b00989eef4e630eed1f8b7c5894cc8641650ab16e6d00c3aa83fa1 -2.452228e+07   
04c852da428c2624